For more analysis <br>
http://people.csail.mit.edu/indyk/6.838-old/handouts/lec17.pdf

In [9]:
import numpy as np
import pandas as pd

In [20]:
# below is 1D 
# assume all values are distinct

def closest_pair_split(arr1, arr2): # sorted arrays in advance with mergesort
    out = []
    delta = None
    c_pair = None
    i = 0
    j = 0
    
    while i < len(arr1) and j < len(arr2):
        delta_c = np.abs(arr1[i] - arr2[j])
        if delta == None or delta > delta_c:
            delta = delta_c
            c_pair = (arr1[i], arr2[j])
        
        if arr1[i] < arr2[j]:
            out.append(arr1[i])
            i += 1
        else:
            out.append(arr2[j])
            j += 1
    
    while i < len(arr1):
        out.append(arr1[i])
        i += 1
    
    while j < len(arr2):
        out.append(arr2[j])
        j += 1
    
    return (out, delta, c_pair)
    


def closest_pair_1D(arr):  # assume len(arr) is power of 2
    if len(arr) == 2:
        if arr[0] < arr[1]:
            out = arr
            delta = np.abs(arr[0] - arr[1])
            c_pair = (arr[0], arr[1])
        
        elif arr[0] > arr[1]:
            out = [arr[1], arr[0]]
            delta = np.abs(arr[0] - arr[1])
            c_pair = (arr[0], arr[1])
            
        return (out, delta, c_pair)
    
    if len(arr) > 2:
        mid = len(arr) // 2
        lefthalf = arr[:mid]
        righthalf = arr[mid:]
        out_l, delta_l, c_pair_l = closest_pair_1D(lefthalf)
        out_r, delta_r, c_pair_r = closest_pair_1D(righthalf)
        out, delta_m, c_pair_split = closest_pair_split(out_l, out_r)
        
        if delta_l <= delta_r:
            if delta_l <= delta_m:
                delta = delta_l
                c_pair = c_pair_l
            else:
                delta = delta_m
                c_pair = c_pair_split
        
        else:
            if delta_r <= delta_m:
                delta = delta_r
                c_pair = c_pair_r
            else:
                delta = delta_m
                c_pair = c_pair_split
        
        return (out, delta, c_pair)

In [21]:
#closest_pair_1D([4,2])
closest_pair_1D([2,4,6,8,9,12,14,16])

([2, 4, 6, 8, 9, 12, 14, 16], 1, (8, 9))

In [5]:
# below is 2D
# assume (for convenience) all points have distinct x-coordinates, distinct y-coordinates(closest_pair_2D)
# point is tuple (..,..)
# out is list [...]
# c_distance is float
# c_pair is ((..,..), (..,..))

def distance(point1, point2): # assume point is a tuple
    distance = np.sqrt((point1[0] - point2[0])**2 + (point1[1] - point2[1])**2)
    return distance

In [6]:
def brute_force(arr): # arr is a list of tuples representing points, len(arr) <= 3
    out_x = [] # sorted array according to x-coordinates
    out_y = [] # sorted array according to y-coordinates
    c_dist = None
    c_pair = None
    
    for i in range(len(arr)):
        for j in range(i+1, len(arr)):
            d = distance(arr[i], arr[j])
            if c_dist == None or c_dist > d:
                c_dist = d
                c_pair = (arr[i], arr[j])
    
    out_x = sorted(arr)
    out_y = sorted(arr, key=lambda x:x[1])
    
    return (out_x, out_y, c_dist, c_pair)

### ClosestPair of a set of points:

Divide the set into two equal sized parts by the line l, and recursively compute the minimal distance in each part.<br>
Let d be the minimal of the two minimal distances.<br>
Eliminate points that lie farther than d apart from l<br>
Sort the remaining points according to their y-coordinates<br>
Scan the remaining points in the y order and compute the distances of each point to its five neighbors.<br>
If any of these distances is less than d then update d.<br>
### Steps 2-6 define the merging process which must be repeated logn times because this is a divide and conquer algortithm:
Step 2 takes O(1) time<br>
Step 3 takes O(n) time<br>
Step 4 is a sort that takes O(nlogn) time<br>
Step 5 takes O(n) time (as we saw in the previous section)<br>
Step 6 takes O(1) time<br>
Hence the merging of the sub-solutions is dominated by the sorting at step 4, and hence takes O(nlogn) time.<br>
This must be repeated once for each level of recursion in the divide-and-conquer algorithm,<br>

In [158]:
# 2018-09-26
# https://www.cs.mcgill.ca/~cs251/ClosestPair/ClosestPairDQ.html
# https://www.cs.mcgill.ca/~cs251/ClosestPair/proofbox.html
# assume points with x, y value as both positive (the upper right quarter of x cross y axis)
# below run time is O(n*logn*logn)


def distance(point1, point2): # assume point is a tuple
    distance = np.sqrt((point1[0] - point2[0])**2 + (point1[1] - point2[1])**2)
    return distance


def merge_sort_by_x(arr): # sort an array of points by x value
    if len(arr) > 1:
        mid = len(arr) // 2
        lefthalf = arr[:mid]
        righthalf = arr[mid:]
        merge_sort_by_x(lefthalf)
        merge_sort_by_x(righthalf)
        
        k = 0
        i = 0
        j = 0
        
        while i < len(lefthalf) and j < len(righthalf):
            if lefthalf[i][0] <= righthalf[j][0]:
                arr[k] = lefthalf[i]
                i += 1
            else:
                arr[k] = righthalf[j]
                j += 1
            k += 1
            
        while i < len(lefthalf):
            arr[k] = lefthalf[i]
            i += 1
            k += 1
        
        while j < len(righthalf):
            arr[k] = righthalf[j]
            j += 1
            k += 1


def merge_sort_by_y(arr): # sort an array of points by y value
    if len(arr) > 1:
        mid = len(arr) // 2
        lefthalf = arr[:mid]
        righthalf = arr[mid:]
        merge_sort_by_y(lefthalf)
        merge_sort_by_y(righthalf)
        
        k = 0
        i = 0
        j = 0
        
        while i < len(lefthalf) and j < len(righthalf):
            if lefthalf[i][1] <= righthalf[j][1]:
                arr[k] = lefthalf[i]
                i += 1
            else:
                arr[k] = righthalf[j]
                j += 1
            k += 1
            
        while i < len(lefthalf):
            arr[k] = lefthalf[i]
            i += 1
            k += 1
        
        while j < len(righthalf):
            arr[k] = righthalf[j]
            j += 1
            k += 1
            

# get the closest pair in strip
# piggyback on merge sort by y
def strip_closest_pair(arr_l, arr_r, pair, d):   
    c_pair = pair
    c_dist = d
    arr = arr_l + arr_r
    merge_sort_by_y(arr)
    # only the negative or positive y difference less than c_dist to a point needs to be considered
    for i, p1 in enumerate(arr):
        for p2 in arr[i+1:]: 
            if (p1 in arr_l and p2 in arr_r) or (p1 in arr_r and p2 in arr_l):
                if p2[1] - p1[1] < d:
                    dist = distance(p1, p2)
                    if dist < d:
                        c_pair = (p1, p2)
                        c_dist = dist
                else:
                    break # compute at mostly 6 points from another array
    return c_pair, c_dist


def closest_pair_2d_rec(arr): # array is sorted by x
    if len(arr) == 1:
        return None, 999999
    elif len(arr) == 2:
        return arr, distance(arr[0], arr[1])
    else:
        mid = len(arr) // 2
        lefthalf = arr[:mid]
        righthalf = arr[mid:]
        l = (lefthalf[-1][0] + righthalf[0][0]) / 2 # l is the middle line across x axis to seperate points
        c_pair_left, c_dist_left = closest_pair_2d_rec(lefthalf)
        c_pair_right, c_dist_right = closest_pair_2d_rec(righthalf)
        c_dist = min(c_dist_left, c_dist_right)
        c_pair = c_pair_left if c_dist == c_dist_left else c_pair_right
        # only the points within d to l needs be compared in strip
        strip_l = []
        strip_r = []
        for p in reversed(lefthalf):
            if p[0] > l - c_dist:
                strip_l.append(p)
            else:
                break
        for p in righthalf:
            if p[0] < l + c_dist:
                strip_r.append(p)
            else:
                break
        c_pair, c_dist = strip_closest_pair(strip_l, strip_r, c_pair, c_dist)
        return c_pair, c_dist
    
def closest_pair_2d(arr):
    merge_sort_by_x(arr)
    closest_pair, closest_distance = closest_pair_2d_rec(arr)
    return arr, closest_pair, closest_distance 

In [159]:
points = [(1, 1), (7, 1.5), (3, 1.5), (2.8, 8), (4, 3), (3, 4), (5, 5), (3.8, 1), (7.3, 5), (6.6, 5.5)]

In [160]:
closest_pair_2d(points)

([(1, 1),
  (2.8, 8),
  (3, 1.5),
  (3, 4),
  (3.8, 1),
  (4, 3),
  (5, 5),
  (6.6, 5.5),
  (7, 1.5),
  (7.3, 5)],
 ((7.3, 5), (6.6, 5.5)),
 0.8602325267042628)

### Improving the algorithm

Step 1: Divide the set into..., and recursively compute the distance in each part, returning the points in each set in sorted order by y-coordinate.<br>
Step 4: Merge the two sorted lists into one sorted list in O(n) time.<br>
Hence the merging process is now dominated by the linear time steps thereby yielding an O(nlogn) algorithm for finding the closest pair of a set of points in the plane. 

In [205]:
def merge_sort_by_x_and_y(arr): # return a tuple of two arrays each sorted by x and y
    if len(arr) == 1:
        return arr, arr
    else:
        mid = len(arr) // 2
        lefthalf = arr[:mid]
        righthalf = arr[mid:]
        
        arr_x_left, arr_y_left = merge_sort_by_x_and_y(lefthalf)
        arr_x_right, arr_y_right = merge_sort_by_x_and_y(righthalf)

        arr_x = []
        arr_y = []
        i_x = 0
        j_x = 0
        i_y = 0
        j_y = 0
        
        while i_x < len(arr_x_left) and j_x < len(arr_x_right):
            if arr_x_left[i_x][0] <= arr_x_right[j_x][0]:
                arr_x.append(arr_x_left[i_x])
                i_x += 1
            else:
                arr_x.append(arr_x_right[j_x])
                j_x += 1
        
        while i_x < len(arr_x_left):
            arr_x.append(arr_x_left[i_x])
            i_x += 1
        while j_x < len(arr_x_right):
            arr_x.append(arr_x_right[j_x])
            j_x += 1   
        
        while i_y < len(arr_y_left) and j_y < len(arr_y_right):
            if arr_y_left[i_y][1] <= arr_y_right[j_y][1]:
                arr_y.append(arr_y_left[i_y])
                i_y += 1
            else:
                arr_y.append(arr_y_right[j_y])
                j_y += 1
        while i_y < len(arr_y_left):
            arr_y.append(arr_y_left[i_y])
            i_y += 1
        while j_y < len(arr_y_right):
            arr_y.append(arr_y_right[j_y])
            j_y += 1
        
        return arr_x, arr_y

    
def strip_closest_pair(arr_l, arr_r, pair, d): # arr_l and arr_r are already sorted by y
    c_pair = pair
    c_dist = d
    
    for p in arr_l:
        candidates = [pp for pp in arr_r if abs(pp[1] - p[1]) < d] # at most 5 these points with y within d of p
        for c in candidates:
            dist = distance(p, c)
            if dist < d:
                c_pair = (p, c)
                c_dist = dist
    return c_pair, c_dist


def closest_pair_2d_rec(arr_x, arr_y): # arr_x, arr_y are already sorted arr by x and y
    if len(arr_x) == 1:
        return None, 999999
    elif len(arr_x) == 2:
        return (arr_x[0], arr_x[1]), distance(arr_x[0], arr_x[1])
    else:
        mid = len(arr_x) // 2
        lefthalf_x = arr_x[:mid]
        lefthalf_y = [p for p in arr_y if p in lefthalf_x]
        righthalf_x = arr_x[mid:]
        righthalf_y = [p for p in arr_y if p in righthalf_x]
        l = (lefthalf_x[-1][0] + righthalf_x[0][0]) / 2 # l is the middle line across x axis to seperate points
        c_pair_left, c_dist_left = closest_pair_2d_rec(lefthalf_x, lefthalf_y)
        c_pair_right, c_dist_right = closest_pair_2d_rec(righthalf_x, righthalf_y)
        c_dist = min(c_dist_left, c_dist_right)
        c_pair = c_pair_left if c_dist == c_dist_left else c_pair_right
        # only the points within d to l needs be compared in strip
        strip_l = []
        strip_r = []
        for p in reversed(lefthalf_x):
            if p[0] > l - c_dist:
                strip_l.append(p)
            else:
                break
        for p in righthalf_x:
            if p[0] < l + c_dist:
                strip_r.append(p)
            else:
                break
        strip_left = [p for p in lefthalf_y if p in strip_l]
        strip_right = [p for p in righthalf_y if p in strip_r]
        c_pair, c_dist = strip_closest_pair(strip_left, strip_right, c_pair, c_dist)
        return c_pair, c_dist

    
def closest_pair_2d(arr):
    arr_x, arr_y = merge_sort_by_x_and_y(arr)
    c_pair, c_dist = closest_pair_2d_rec(arr_x, arr_y)
    return arr_x, arr_y, c_pair, c_dist

In [208]:
_, _, closest_pair, closest_distance = closest_pair_2d(points)
closest_pair, closest_distance

(((6.6, 5.5), (7.3, 5)), 0.8602325267042628)